# Assessment of Wind Models and Optimization Methods

In [29]:
from flightdata import Flight
from flightanalysis import Box, Section
from flightanalysis.environment.wind import fit_wind, WindModelBuilder
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from json import dumps
import plotly
plotly.offline.init_notebook_mode()

In [23]:
flight = Flight.from_csv("data/cg_changes/baseline.csv")
box = Box.from_f3a_zone("data/cg_changes/box.f3a")
sec = Section.from_flight(flight, box).flying_only()


In [21]:
from time import time

def tfunc(*args, **kwargs):
    s = time()
    res = fit_wind(*args, **kwargs)
    return dict(time=time()-s, heading=res.args[0], speed=res.args[1], exponent=res.args[2])
    

opt_vars = dict(
    b_defaul = tfunc(sec, WindModelBuilder.power_law(), bounds=True),
    ubdefaul = tfunc(sec, WindModelBuilder.power_law(), bounds=False),
    b__nmead = tfunc(sec, WindModelBuilder.power_law(), bounds=True, method="nelder-mead"),
    ub_nmead = tfunc(sec, WindModelBuilder.power_law(), bounds=False, method="nelder-mead"),
    b_powell = tfunc(sec, WindModelBuilder.power_law(), bounds=True, method="Powell"),
    ubpowell = tfunc(sec, WindModelBuilder.power_law(), bounds=False, method="Powell"),
)

print(dumps(opt_vars))



{
    "b_defaul": {
        "time": 3.948045492172241,
        "heading": 0.0,
        "speed": 8.165812502780083,
        "exponent": 0.3745634430310281
    },
    "ubdefaul": {
        "time": 20.720499515533447,
        "heading": 6.241231748750687,
        "speed": 8.46300625050055,
        "exponent": 0.4005644918523894
    },
    "b__nmead": {
        "time": 18.536996364593506,
        "heading": 0.0,
        "speed": 8.165530363098696,
        "exponent": 0.37449152640687444
    },
    "ub_nmead": {
        "time": 9.362324714660645,
        "heading": 6.241226422162353,
        "speed": 8.462996559566179,
        "exponent": 0.4005606870565993
    },
    "b_powell": {
        "time": 14.622764825820923,
        "heading": 6.241301942457775,
        "speed": 8.4639383246321,
        "exponent": 0.40052132988263267
    },
    "ubpowell": {
        "time": 11.598246574401855,
        "heading": 6.24122254341178,
        "speed": 8.463002083418946,
        "exponent": 0.4005038127

In [24]:
wind_models = {}
wind_models["power law"] = fit_wind(sec, WindModelBuilder.power_law(), bounds=False, method="nelder-mead")

In [25]:
wind_models["cubic spline"] = fit_wind(sec, WindModelBuilder.fit(
    minh=0, 
    maxh=max(sec.gpos.z),
    npoints=5,
    kind="cubic",
    fill_value="extrapolate"
), bounds=False, method="nelder-mead")

In [26]:
wind_models["linear"] = fit_wind(sec, WindModelBuilder.fit(
    minh=0, 
    maxh=max(sec.gpos.z),
    npoints=5,
    kind="linear",
    fill_value="extrapolate"
))

In [27]:
wind_models["slinear"] = fit_wind(sec, WindModelBuilder.fit(
    minh=0, 
    maxh=max(sec.gpos.z),
    npoints=5,
    kind="linear",
    fill_value="extrapolate"
), bounds=False, method="nelder-mead")

In [28]:
maxh=max(sec.gpos.z)
heights = np.linspace(0, max(sec.gpos.z), 100)

fig =go.Figure()

for key, val in wind_models.items():

    fig.add_trace(go.Scatter(
        x=abs(val(heights, np.zeros(len(heights)))), 
        y=heights,
        name=f"{key}, heading = {np.degrees(val.args[0]):.0f}"
    ))

fig.update_layout(
    height=800, 
    width=800, 
    title="Comparison of Wind Models",
    legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
)
